# Loading libraries and data

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df_train = pd.read_csv('event-log-training.csv')

In [3]:
df_train

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp
0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000
1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000
2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000
3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000
4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000
...,...,...,...,...,...,...,...,...
36623,42949673754,18,Simulated process instance,Print,PRN1,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000
36624,42949673755,18,Simulated process instance,Print,PRN1,Fusing,complete,25-12-1970 16:05:00.000
36625,42949673756,18,Simulated process instance,Print,PRN1,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000
36626,42949673757,18,Simulated process instance,Print,PRN1,Erase Charge on Drum,complete,25-12-1970 16:22:00.000


# Convert timestamp to datetime

In [4]:
date_list = []

for time in df_train['event time:timestamp']:
    datex = time[:-4]
    date = datetime.strptime(datex, '%d-%m-%Y %H:%M:%S')

    date_list.append(date)

In [5]:
df_train['time and date'] = date_list

In [6]:
df_train

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date
0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000,1970-01-01 01:00:00
1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000,1970-01-01 01:15:00
2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000,1970-01-01 01:26:00
3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000,1970-01-01 01:38:00
4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000,1970-01-01 01:51:00
...,...,...,...,...,...,...,...,...,...
36623,42949673754,18,Simulated process instance,Print,PRN1,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000,1970-12-25 15:51:00
36624,42949673755,18,Simulated process instance,Print,PRN1,Fusing,complete,25-12-1970 16:05:00.000,1970-12-25 16:05:00
36625,42949673756,18,Simulated process instance,Print,PRN1,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000,1970-12-25 16:10:00
36626,42949673757,18,Simulated process instance,Print,PRN1,Erase Charge on Drum,complete,25-12-1970 16:22:00.000,1970-12-25 16:22:00


# Add in true next event and time it takes for case 4

In [7]:
df_train['event lifecycle:transition'].value_counts()

complete    33802
start        2826
Name: event lifecycle:transition, dtype: int64

In [8]:
dct_events = dict(df_train['event concept:name'].value_counts())

In [9]:
df_train['case concept:name'].value_counts()

12    2140
33    1594
42    1390
24    1285
6     1198
      ... 
27      62
80      59
56      55
51      43
4       43
Name: case concept:name, Length: 80, dtype: int64

In [10]:
df_train.groupby(by = ['case concept:name', 'time and date'], as_index = False)
df_case4 = df_train[df_train['case concept:name'] == 4]

In [11]:
#df_case4

Add event column

In [12]:
event_lst4 = [event for event in df_case4['event concept:name']]

event_lst4 = event_lst4[1:]
event_lst4.append('-')
    

In [13]:
df_case4['next_event'] = event_lst4

<ipython-input-13-fa9b75893eb4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['next_event'] = event_lst4


Add time to next event column

In [14]:
nexttime_lst1 = [time for time in df_case4['time and date']]

nexttime_lst = nexttime_lst1[1:]
nexttime_lst.append(nexttime_lst[-1])

In [15]:
time_diff = []

for i in range(len(nexttime_lst)):
    time_diff.append(nexttime_lst[i] - nexttime_lst1[i])

In [16]:
df_case4['time to next event'] = time_diff

<ipython-input-16-6e0ba6d54076>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['time to next event'] = time_diff


In [17]:
df_case4

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date,next_event,time to next event
64,146028888064,4,Simulated process instance,Copy/Scan,PRN1,Job,start,01-01-1970 11:23:00.000,1970-01-01 11:23:00,Copy/Scan,271 days 01:10:00
26983,146028888065,4,Simulated process instance,Copy/Scan,PRN1,Copy/Scan,complete,29-09-1970 12:33:00.000,1970-09-29 12:33:00,Collect Copy/Scan Options,0 days 00:18:00
26984,146028888066,4,Simulated process instance,Copy/Scan,PRN1,Collect Copy/Scan Options,complete,29-09-1970 12:51:00.000,1970-09-29 12:51:00,Place Doc,0 days 00:08:00
26985,146028888067,4,Simulated process instance,Copy/Scan,PRN1,Place Doc,complete,29-09-1970 12:59:00.000,1970-09-29 12:59:00,Illuminate Document,0 days 00:19:00
26986,146028888068,4,Simulated process instance,Copy/Scan,PRN1,Illuminate Document,complete,29-09-1970 13:18:00.000,1970-09-29 13:18:00,Move Scan Head,0 days 00:22:00
26987,146028888069,4,Simulated process instance,Copy/Scan,PRN1,Move Scan Head,complete,29-09-1970 13:40:00.000,1970-09-29 13:40:00,Focus Light Beam,0 days 00:18:00
26988,146028888070,4,Simulated process instance,Copy/Scan,PRN1,Focus Light Beam,complete,29-09-1970 13:58:00.000,1970-09-29 13:58:00,A/D Conversion,0 days 00:07:00
26989,146028888071,4,Simulated process instance,Copy/Scan,PRN1,A/D Conversion,complete,29-09-1970 14:05:00.000,1970-09-29 14:05:00,Interpolation,0 days 00:09:00
26990,146028888072,4,Simulated process instance,Copy/Scan,PRN1,Interpolation,complete,29-09-1970 14:14:00.000,1970-09-29 14:14:00,Filtered Image,0 days 00:11:00
26991,146028888073,4,Simulated process instance,Copy/Scan,PRN1,Filtered Image,complete,29-09-1970 14:25:00.000,1970-09-29 14:25:00,Collect Image,0 days 00:19:00


# Next event and time column for all data


In [18]:
cases = list(df_train['case concept:name'].unique())

In [19]:
df_result = pd.DataFrame()

In [20]:
#event column
total_events = []
for case in cases: 
    df_case = df_train[df_train['case concept:name'] == case].copy()
    
    #create next event list
    event_lst = [event for event in df_case['event concept:name']]
    event_lst = event_lst[1:]
    event_lst.append('-')
    
    #create next time list
    nexttime_lst1 = [time for time in df_case['time and date']]
    nexttime_lst = nexttime_lst1[1:]
    nexttime_lst.append(nexttime_lst[-1])
    
    #create difference list
    time_diff = []
    for i in range(len(nexttime_lst)):
        time_diff.append(nexttime_lst[i] - nexttime_lst1[i])
    
    #append columns to df case
    df_case['Time to next event'] = time_diff
    df_case['Next event'] = event_lst
    
    #append df case to final df
    df_result = df_result.append(df_case)

Prediction stuff

In [22]:
lst = df_result[['event concept:name', 'Next event']].apply(tuple, axis=1)

In [23]:
lst = [elm for elm in lst]

In [24]:
count_dict = {}
for elm in lst:
    if elm in count_dict:
        count_dict[elm] +=1
    else:
        count_dict[elm] = 1

In [25]:
count_dict

{('Job', 'Remote Print'): 44,
 ('Remote Print', 'Read Print Options'): 44,
 ('Read Print Options', 'Rasterization'): 44,
 ('Rasterization', 'Interpretation'): 44,
 ('Interpretation', 'Unformatted Text'): 34,
 ('Unformatted Text', 'Interpretation'): 32,
 ('Interpretation', 'Rendering'): 75,
 ('Rendering', 'Screening'): 145,
 ('Screening', 'Interpretation'): 249,
 ('Interpretation', 'FM Screening'): 81,
 ('FM Screening', 'Unformatted Text'): 5,
 ('Unformatted Text', 'Screening'): 27,
 ('Interpretation', 'Current Page Image'): 111,
 ('Current Page Image', 'Interpretation'): 110,
 ('Interpretation', 'Accumulate Images'): 84,
 ('Accumulate Images', 'Rendering'): 192,
 ('Rendering', 'Unformatted Text'): 13,
 ('Interpretation', 'AM Screening'): 51,
 ('AM Screening', 'Interpretation'): 57,
 ('Screening', 'Current Page Image'): 249,
 ('Accumulate Images', 'Interpretation'): 40,
 ('Interpretation', 'Screening'): 201,
 ('Screening', 'Unformatted Text'): 21,
 ('Unformatted Text', 'Current Page Ima

In [ ]:
df_train[df_train['eventID '] == 146028888106]

In [ ]:
df_train.head()